**TruncatedSVD** implements a variant of singular value decomposition *(SVD)* that only computes the  largest singular values, where  is a user-specified parameter.

Mathematically, truncated *SVD* Applied to traning samples $X$ produces a low.rank approcimation $X$:
$$X\approx X_K = U_k\Sigma_k V_k^T$$
After the operation, $U_k \Sigma_k$ is the transformed traning set with *k* features (called *n_components* in the API)
To also transform a test set $X$, we multiply it with $V_k$:
$$X' = XV_k$$

In [2]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]

dataset = fetch_20newsgroups(
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
)

labels = dataset.target
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]

print(f"{len(dataset.data)} documents - {true_k} categories")

ModuleNotFoundError: No module named 'numpy'